<a href="https://colab.research.google.com/github/cheangjialiang/forexforcast/blob/master/multi_step_forcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install "/content/drive/My Drive/Colab Notebooks/lib/TA_Lib-0.4.18-cp36-cp36m-linux_x86_64.whl"
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
#!pip install jupyterlab
#!npm i -g npm
#!npm install -g localtunnel
#!whereis lt

In [0]:
PAIRS_LIST        = ["EURUSD", "USDJPY", "EURJPY"] #, "USDCHF", "EURCHF"] #, "AUDUSD" ]
PREDICTING_PAIR   = "EURUSD" 
PREDICTING_COLUMN = "close"
LOOK_BACK         = 32 # 30 * 15 = 8 hour
SPLIT             = 0.95 # data split ration for training and testing
DATA_PATH          = "/content/drive/My Drive/Colab Notebooks/data/"
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/My Drive/Colab Notebooks/data/USDCHF-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/EURJPY-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/USDCAD-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/AUDUSD-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/USDJPY-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/EURUSD-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/EURCHF-2000-2020-15m.csv
/content/drive/My Drive/Colab Notebooks/data/weights.best.hdf5


In [0]:
#from IPython.lib import passwd
#password = passwd("secret")
#password
#!npm install -g localtunnel
#get_ipython().system_raw('jupyter notebook --ip=0.0.0.0 --port=8888 &')
#get_ipython().system_raw('jupyter lab --ip=0.0.0.0 --port=8889 &')
#get_ipython().system_raw('lt --port 8888 >> url.txt 2>&1 &')
#get_ipython().system_raw('lt --port 8888 --ip="172.28.0.2" >> url.txt 2>&1 &')
#!cat url.txt


In [0]:
jupyter_running = !jupyter notebook list | grep 8888
if not jupyter_running:
  !mkdir -p /content/.jupyter
  !echo '{ "NotebookApp": { "password": "sha1:975a59cda9c4:e148fb36c9433c39fb5e678e33a45215fc83aa27" } }' > /content/.jupyter/jupyter_notebook_config.json
  get_ipython().system_raw('jupyter lab &')

In [0]:
!cp '/content/drive/My Drive/Colab Notebooks/util/TechnicalIndicators.py' .
!pip install alpha_vantage

In [0]:
import pandas as pd
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import keras
from TechnicalIndicators import TechnicalIndicators
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
TI=TechnicalIndicators()
close_data=TI.close_data[['4. close']]
macd_data=TI.macd_data
rsi_data=TI.rsi_data
bbands_data=TI.bbands_data
dataset = pd.concat([macd_data,rsi_data,bbands_data,close_data], axis=1,sort=True).reindex(macd_data.index)
dataset=dataset.drop(dataset.index[len(dataset)-1])
close_data = dataset[['4. close']]

Using TensorFlow backend.


Enter stock name:AAPL


In [0]:
X = dataset.drop(dataset.index[len(dataset)-1])
y = close_data.drop(close_data.index[0])
values_x = X.values
values_y = y.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data_X = scaler.fit_transform(values_x)
scaled_data_y = scaler.fit_transform(values_y)
X_train = scaled_data_X[:int(X.shape[0]*0.8)]
X_test = scaled_data_X[int(y.shape[0]*0.8):]
y_train = scaled_data_y[:int(X.shape[0]*0.8)]
y_test = scaled_data_y[int(y.shape[0]*0.8):]

In [0]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
y_train = y_train.reshape(y_train.shape[0],1,1)
y_test = y_test.reshape(y_test.shape[0],1,1)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(3997, 1, 8) (3997, 1, 1) (1000, 1, 8) (1000, 1, 1)


In [0]:
a_train,b_train,c_test,d_test =[],[],[],[]
for i in range(65,y_train.shape[0]):
    a_train.append(X_train[i-65:i-5,0])
    b_train.append(y_train[i-5:i,0])
for x in range(65,y_test.shape[0]):
    c_test.append(X_test[x-65:x-5,0])
    d_test.append(y_test[x-5:x,0])
a_train, b_train,c_test,d_test = np.array(a_train), np.array(b_train),np.array(c_test),np.array(d_test)
print(a_train.shape,b_train.shape,c_test.shape,d_test.shape)

(3932, 60, 8) (3932, 5, 1) (935, 60, 8) (935, 5, 1)


In [0]:
a_train = np.reshape(a_train, (a_train.shape[0],a_train.shape[1],a_train.shape[2]))
b_train = np.reshape(b_train,(b_train.shape[0],b_train.shape[1]))
c_test = np.reshape(c_test,(c_test.shape[0],c_test.shape[1],c_test.shape[2]))
d_test = np.reshape(d_test,(d_test.shape[0],d_test.shape[1]))
print(a_train.shape,b_train.shape,c_test.shape,d_test.shape)

(3932, 60, 8) (3932, 5) (935, 60, 8) (935, 5)


In [0]:
OUTPUT_BEST_WEIGHT = DATA_PATH+"weights.best.hdf5"
# Save the best weight during training.
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(OUTPUT_BEST_WEIGHT, monitor='val_mse', verbose=1, save_best_only=True, mode='min')

# Monitor the trianing progress via TensorBoard
# log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard = TensorBoard(log_dir=log_dir)

# Callbacks
callbacks_list = [checkpoint] # , tensorboard]
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
def scheduler(epoch):
    if epoch%10==0 and epoch!=0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr*.9)
        print("lr changed to {}".format(lr*.9))
    return K.get_value(model.optimizer.lr)
# To improve the weights towards the global optimal, I retrained the model with LearningRateScheduler added
lr_decay = LearningRateScheduler(scheduler)
callbacks_list = [checkpoint, lr_decay] # , tensorboard]

In [0]:
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(a_train.shape[1],a_train.shape[2])))
model.add(LSTM(units=64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(5))
ADAM=keras.optimizers.Adam(0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=ADAM)
history = model.fit(a_train,b_train,epochs=50,batch_size=60,validation_data=(c_test,d_test),verbose=1,shuffle=False)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [0]:
yhat = model.predict(c_test)

In [0]:
print("yhat:",yhat.shape, "  X_test:", X_test.shape, "  y_test:", y_test.shape)

yhat: (935, 5)   X_test: (1000, 1, 8)   y_test: (1000, 1, 1)


In [0]:
scaler.inverse_transform(yhat).shape

(935, 5)

In [0]:
yhat = model.predict(c_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[2]))

inv_yhat = concatenate((yhat, X_test[60:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

y_test = y_test.reshape((len(y_test), 1))
inv_y = concatenate((y_test[60:],X_test[60:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)




ValueError: ignored